This implementation follows the basic steps of the XGBoost algorithm, including:

Initializing the model predictions to the mean of the training labels
Looping over a fixed number of trees, and for each tree:
a. Computing the gradient and hessian of the loss function for the current predictions
b. Fitting a decision tree to the negative gradient of the loss function
c. Updating the model predictions with the current tree's weighted predictions
Computing the final model predictions and evaluating the performance.
Note that this is a simplified implementation and does not include all the advanced features and optimizations of the XGBoost algorithm found in more complete implementations such as the XGBoost library for Python.

In [ ]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Load the Boston Housing dataset
boston = load_boston()

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.2, random_state=123)

# Define the XGBoost model parameters
n_trees = 10
max_depth = 5
learning_rate = 0.1
reg_lambda = 1

# Initialize the model predictions to the mean of the training labels
mean_y = np.mean(y_train)
pred_train = np.full((len(y_train),), mean_y)
pred_test = np.full((len(y_test),), mean_y)

# Loop over the number of trees
for i in range(n_trees):
    # Compute the gradient and hessian of the loss function for the current predictions
    grad = y_train - pred_train
    hess = np.ones_like(y_train)

    # Fit a decision tree to the negative gradient of the loss function
    tree = DecisionTreeRegressor(max_depth=max_depth)
    tree.fit(X_train, -grad)

    # Compute the predictions for the training and testing sets
    pred_train += learning_rate * tree.predict(X_train)
    pred_test += learning_rate * tree.predict(X_test)

    # Apply L2 regularization to the tree weights
    tree_weight = reg_lambda / (1.0 + reg_lambda * i)

    # Update the model predictions with the current tree's weighted predictions
    pred_train = (1 - tree_weight) * pred_train + tree_weight * pred_test
    pred_test = (1 - tree_weight) * pred_test + tree_weight * tree.predict(X_test)

# Compute the final model predictions and evaluate the performance
y_pred = pred_test
mse = mean_squared_error(y_test, y_pred)
print("XGBoost MSE:", mse)
